<a href="https://colab.research.google.com/github/Fripe070/Stable-Diffusion-WebUI-Colab/blob/main/Stable_Diffusion_WebUI_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install


## General setup


In [ ]:
#@markdown ## Initial setup

import os
from google.colab.output import eval_js
os.environ['colab_url'] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")

!apt -qq -y update &>/dev/null
!wget -nv http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget -nv https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget -nv https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget -nv https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt -qq install libunwind8-dev &>/dev/null
!dpkg -i *.deb &>/dev/null
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!apt -qq -y install aria2 &>/dev/null
!apt -qq install unzip &>/dev/null
!pip install -q xformers==0.0.16
!pip install -q triton==2.0.0
!pip install -q requests


In [ ]:
#@markdown ## Download WebUI

#!git clone -b v2.0 https://github.com/camenduru/stable-diffusion-webui
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd /content/stable-diffusion-webui


In [ ]:
#@markdown ## Fix stuff
# I have absolutely 0 clue why this is here but it's probably important lmao
!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py


In [ ]:
#@markdown ## Define helper functions
import json
import re
import shlex
import subprocess
import requests
from pathlib import Path
from typing import Union
from urllib.parse import unquote_plus, urlparse

config_path = Path("/content/stable-diffusion-webui/config.json")
ui_config_path = Path("/content/stable-diffusion-webui/ui-config.json")


def load_config() -> dict:
    with open(config_path, "r") as file:
        return json.load(file)


def save_config(config: dict, /) -> None:
    with open(config_path, "w") as file:
        json.dump(config, file)


def load_sd_config() -> dict:
    with open(ui_config_path, "r") as file:
        return json.load(file)


def save_ui_config(ui_config: dict, /) -> None:
    with open(ui_config_path, "w") as file:
        json.dump(ui_config, file)


class Model:
    def __init__(
            self,
            *,
            name: str,
            filename: str,
            download_url: str
        ) -> None:
        self.name = name
        self.filename = filename
        self.download_url = download_url


    def download(self, models_path: Union[Path, str], *, name: Union[str, None] = None) -> None:
        filename = name.strip().replace(" ", "-") if name else self.filename
        save_to = Path(models_path) / filename
        cmd = shlex.split(
            (
                f'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M '
                f'"{self.download_url}" '
                f'-d "{save_to.parent}" '
                f'-o "{save_to.name}"'
            )
        )

        print(f"Saving model to: {save_to}")

        proccess = subprocess.run(cmd)
        if proccess.returncode != 0:
            raise RuntimeError(f"Failed to download model: {self.filename}")

        print("Model downloaded")


class CivitaiModel(Model):
    def __init__(
            self,
            *,
            id: int,
            name: str,
            filename: str,
            download_url: str
        ) -> None:
        super().__init__(name=name, filename=filename, download_url=download_url)
        self.id = id

    @staticmethod
    def _id_from_url(url: str) -> int:
        parsed_url = urlparse(unquote_plus(url))
        if match := re.search(r"models/(\d+)", parsed_url.path):
            return int(match.groups()[0])
        else:
            raise ValueError(f"Can't find model ID in URL: {url}")

    @classmethod
    def from_model_page_url(cls, page_url: str) -> "CivitaiModel":
        model_id = cls._id_from_url(page_url)
        model_data = requests.get(f"https://civitai.com/api/v1/models/{model_id}").json()
        file = model_data["modelVersions"][0]["files"][0]

        return cls(
            id=model_data["id"],
            name=model_data["name"],
            filename=file["name"],
            download_url=file["downloadUrl"]
        )

    @classmethod
    def from_download_url(cls, download_url: str) -> "CivitaiModel":
        model_id = cls._id_from_url(download_url)
        model_data = requests.get(f"https://civitai.com/api/v1/model-versions/{model_id}").json()
        file = model_data["files"][0]

        return cls(
            id=model_data["modelId"],
            name=model_data["model"]["name"],
            filename=file["name"],
            download_url=file["downloadUrl"]
        )


def model_from_url(url: str) -> Union[Model, CivitaiModel]:
    parsed_url = urlparse(unquote_plus(url))

    if parsed_url.hostname == "civitai.com":
        if parsed_url.path.startswith("/api/download/"):
            return CivitaiModel.from_download_url(url)
        elif parsed_url.path.startswith("/models/"):
            return CivitaiModel.from_model_page_url(url)
        
    return Model(
        name=Path(parsed_url.path).stem,
        filename=Path(parsed_url.path).name,
        download_url=url
    )

In [ ]:
#@markdown ## Set default config values
save_ui_config(
    {
        "txt2img/Sampling steps/value": 50,
        "img2img/Sampling steps/value": 50,
    }
)
save_config(
    {
        "show_progress_every_n_steps": 2,
        "live_preview_refresh_period": 20.0,
    }
)


In [ ]:
#@markdown ## Download extensions

# Extensions
!wget -q https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /content/stable-diffusion-webui/scripts/run_n_times.py
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser -q
!git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/sd-webui-additional-networks -q
!git clone https://github.com/camenduru/sd-webui-tunnels /content/stable-diffusion-webui/extensions/sd-webui-tunnels -q

# Extensions added by fripe
!git clone https://github.com/alemelis/sd-webui-ar.git /content/stable-diffusion-webui/extensions/sd-webui-ar -q
!git clone https://github.com/toriato/stable-diffusion-webui-wd14-tagger.git /content/stable-diffusion-webui/extensions/stable-diffusion-webui-wd14-tagger -q
!git clone https://github.com/dfaker/SD-latent-mirroring.git /content/stable-diffusion-webui/extensions/SD-latent-mirroring -q


## Other extensions
You may not need/want all these, so you can pick and choose induvidualy.


In [ ]:
#@markdown ### [ControlNet Extension](https://github.com/Mikubill/sd-webui-controlnet)
#@markdown Allows extra control of generated images, by enforcing certain constraints

# Extensions
!git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet -q
!git clone https://github.com/camenduru/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor -q
!git clone https://github.com/jexom/sd-webui-depth-lib /content/stable-diffusion-webui/extensions/sd-webui-depth-lib -q
!git clone https://github.com/hnmr293/posex /content/stable-diffusion-webui/extensions/posex -q

#M Models
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_canny-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_canny-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_depth-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_depth-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_hed-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_hed-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_mlsd-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_mlsd-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_normal-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_normal-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_openpose-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_openpose-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_scribble-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_scribble-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_seg-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_seg-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/hand_pose_model.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o hand_pose_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/body_pose_model.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o body_pose_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/dpt_hybrid-midas-501f0c75.pt -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/midas -o dpt_hybrid-midas-501f0c75.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_large_512_fp32.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_large_512_fp32.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_tiny_512_fp32.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_tiny_512_fp32.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/network-bsds500.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/hed -o network-bsds500.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/upernet_global_small.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/uniformer -o upernet_global_small.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_style_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_style_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_sketch_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_sketch_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_seg_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_seg_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_openpose_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_openpose_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_keypose_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_keypose_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_depth_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_depth_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_color_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_color_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_canny_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_canny_sd14v1.pth


In [ ]:
#@markdown ### [Deforum Extension](https://github.com/deforum-art/deforum-for-automatic1111-webui)
#@markdown Allows for creating interesting animations transitioning between multiple prompts or other parameters. 
!git clone https://github.com/deforum-art/deforum-for-automatic1111-webui /content/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui -q


In [ ]:
#@markdown ### [Booru tag autocompletion](https://github.com/DominikDoom/a1111-sd-webui-tagcomplete)
#@markdown Provides borru tag completion in the prompt input box
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git /content/stable-diffusion-webui/extensions/tag-autocomplete -q


In [ ]:
#@markdown ### [Stable Diffusion Webui Inspiration](https://github.com/yfszzx/stable-diffusion-webui-inspiration)
#@markdown Adds an artstyle moodboard
!git clone https://github.com/yfszzx/stable-diffusion-webui-inspiration.git /content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration -q

# Image dataset
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/yfszzx/inspiration/resolve/main/inspiration.zip -d /content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration/ -o inspiration.zip
!unzip -qoO UTF-8 /content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration/inspiration.zip -d /content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration/
!rm /content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration/inspiration.zip


In [ ]:
#@markdown ### [Dreambooth Extension](https://github.com/d8ahazard/sd_dreambooth_extension)
#@markdown Adds a tab allowing you tu use dreambooth to train models on specific people/objects/styles 
!git clone https://github.com/d8ahazard/sd_dreambooth_extension.git /content/stable-diffusion-webui/extensions/sd_dreambooth_extension -q


In [ ]:
#@markdown ### [Stable Diffusion Civitai Browser](https://github.com/camenduru/sd-civitai-browser)
#@markdown An extension to browse Civitai for models
!git clone -b v2.0 https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/sd-civitai-browser -q


In [ ]:
#@markdown ### [Steps Animation](https://github.com/vladmandic/sd-extension-steps-animation)
#@markdown An extension to create an animation from the sampling steps in your image generation
!git clone https://github.com/vladmandic/sd-extension-steps-animation.git /content/stable-diffusion-webui/extensions/sd-extension-steps-animation -q


In [ ]:
#@markdown ### [Latent Couple](https://github.com/opparco/stable-diffusion-webui-two-shot)
#@markdown Allows including subprompts mapped to different areas of the output image
!git clone https://github.com/opparco/stable-diffusion-webui-two-shot /content/stable-diffusion-webui/extensions/stable-diffusion-webui-two-shot -q


# Download models
You most likely **don't** want to run this entire block, instead pick and choose induvidial models to download.

## VAE

> The VAE encoder compresses the image from pixel space to a smaller dimensional latent space, capturing a more fundamental semantic meaning of the image.

\- [Stable Diffusion Architeture - Wikipedia](https://en.wikipedia.org/wiki/Stable_Diffusion#Architecture)

In [ ]:
#@markdown ### vae-ft-mse-840000-ema-pruned 
#@markdown Recommended för GorynichMix
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o vae-ft-mse-840000-ema-pruned.vae.pt

set_as_default = True  # @param {type: "boolean"}

if set_as_default:
    config = load_config()
    config["sd_vae"] = "vae-ft-mse-840000-ema-pruned.vae.pt"
    save_config(config)


In [ ]:
#@markdown ### nai.vae.pt 
#@markdown Recommended för Abyss OrangeMix
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -d /content/stable-diffusion-webui/models/Stable-diffusion -o nai.vae.pt

set_as_default = True  # @param {type: "boolean"}

if set_as_default:
    config = load_config()
    config["sd_vae"] = "nai.vae.pt"
    save_config(config)


## Diffusion model


In [ ]:
#@markdown ### Stable diffusion 1.5
#@markdown [Model README](https://huggingface.co/runwayml/stable-diffusion-v1-5)\
#@markdown Style: none
model_from_url(
    "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.safetensors"
).download(
    "/content/stable-diffusion-webui/models/Stable-diffusion",
    name="stable_diffusion_v1-5.safetensors"
)

In [ ]:
#@markdown ### GorynichMix
#@markdown [Model README](https://civitai.com/models/4815/gorynichmix)\
#@markdown Style: anime
model_from_url(
    "https://civitai.com/api/download/models/5848"
).download(
    "/content/stable-diffusion-webui/models/Stable-diffusion"
)

In [ ]:
#@markdown ### Pastel Mix
#@markdown [Model README](https://huggingface.co/andite/pastel-mix)\
#@markdown Style: anime
model_from_url(
    "https://huggingface.co/andite/pastel-mix/resolve/main/pastelmix-better-vae.safetensors"
).download(
    "/content/stable-diffusion-webui/models/Stable-diffusion"
)

In [ ]:
#@markdown ### Abyss OrangeMix AOM3A3
#@markdown [Model README](https://huggingface.co/WarriorMama777/OrangeMixs)\
#@markdown Style: anime
model_from_url(
    "https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors"
).download(
    "/content/stable-diffusion-webui/models/Stable-diffusion"
)

In [ ]:
#@markdown ### Realistic Vision V1.4
#@markdown [Model README](https://huggingface.co/SG161222/Realistic_Vision_V1.4)\
#@markdown Style: realistic
model_from_url(
    "https://huggingface.co/SG161222/Realistic_Vision_V1.4/resolve/main/Realistic_Vision_V1.4-pruned-fp16.safetensors"
).download(
    "/content/stable-diffusion-webui/models/Stable-diffusion"
)

In [ ]:
# @markdown ### Custom model

# @markdown The url of the file you want to download
model_url = "https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned.safetensors" # @param {type: "string"}
# @markdown The name of the file to be placed in the model directory, leave blank to have the script figure it out itself.
model_filename = "anything v4.5.safetensors"  # @param {type: "string"}

model_from_url(model_url).download(
    models_path="/content/stable-diffusion-webui/models/Stable-diffusion", 
    name=model_filename
)

## LoRA


In [ ]:
# @markdown ### Download model

# @markdown The url of the file you want to download
model_url = "https://civitai.com/api/download/models/24617" # @param {type: "string"}
# @markdown The name of the file to be placed in the model directory, leave blank to have the script figure it out itself.
model_filename = ""  # @param {type: "string"}

model_from_url(model_url).download(
    models_path="/content/stable-diffusion-webui/models/Lora", 
    name=model_filename
)

## Textual inversion


In [ ]:
# @markdown ### Download embedding

# @markdown The url of the file you want to download
model_url = "https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.safetensors" # @param {type: "string"}
# @markdown The name of the file to be placed in the model directory, leave blank to have the script figure it out itself.
model_filename = ""  # @param {type: "string"}

model_from_url(model_url).download(
    models_path="/content/stable-diffusion-webui/embeddings", 
    name=model_filename
)

# Run WebUI

In [ ]:
#@markdown ## Launch
%cd /content/stable-diffusion-webui
!python launch.py --share --xformers --enable-insecure-extension-access --theme dark --gradio-queue --cloudfl